In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import pyterrier as pt
import numpy as np
import pandas as pd

from tqdm import tqdm
import zipfile
import glob
import ir_datasets
if not pt.started():
  pt.init()

from pyterrier_t5 import MonoT5ReRanker

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_nyt = ir_datasets.load("nyt")

In [4]:
index_ref = pt.IndexRef.of("/app/indices/nyt/")

In [10]:
monoT5 = MonoT5ReRanker(text_field="body", batch_size=100, verbose=True)

bm25 = pt.BatchRetrieve(index_ref , wmodel='BM25', num_results=200)
mono_pipeline = bm25 >> pt.text.get_text(index_ref, "body") >> monoT5

mono_pipeline_500 = pt.BatchRetrieve(index_ref , wmodel='BM25', num_results=500) >>  pt.text.get_text(index_ref, "body") >> monoT5
mono_pipeline_50 = pt.BatchRetrieve(index_ref , wmodel='BM25', num_results=50) >>  pt.text.get_text(index_ref, "body") >> monoT5

/usr/local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


09:40:02.984 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
09:40:03.292 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
09:40:06.126 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
09:40:06.403 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
09:40:06.681 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk


In [6]:
ds1 = pt.get_dataset('irds:nyt/trec-core-2017')

In [7]:
pt.Experiment(
    [bm25],
    ds1.get_topics('title'),
    ds1.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["BM25"],
    verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [00:05<00:00,  5.05s/system]


,name,map,recip_rank,P_10,ndcg_cut_10
0,BM25,0.160819,0.684385,0.458,0.371569


In [8]:
pt.Experiment(
    [mono_pipeline],
    ds1.get_topics('title'),
    ds1.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [03:49<00:00, 229.76s/system]


,name,map,recip_rank,P_10,ndcg_cut_10
0,MonoT5,0.182525,0.752156,0.574,0.468468


In [9]:
pt.Experiment(
    [mono_pipeline_500],
    ds1.get_topics('title'),
    ds1.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [09:31<00:00, 571.10s/system]


,name,map,recip_rank,P_10,ndcg_cut_10
0,MonoT5,0.216186,0.739844,0.566,0.460179


In [11]:
pt.Experiment(
    [mono_pipeline_50],
    ds1.get_topics('title'),
    ds1.get_qrels(),
    eval_metrics=["map", "recip_rank", "P_10", "ndcg_cut_10"],
    names=["MonoT5"],
    verbose=True
)

pt.Experiment: 100%|██████████| 1/1 [00:57<00:00, 57.34s/system]


,name,map,recip_rank,P_10,ndcg_cut_10
0,MonoT5,0.110421,0.728183,0.562,0.450768
